# Image classification of brain tumor

In this project, I train a machine learning model to classify brain scans as having tumors or not.

Brain tumor classification using image data is an important area of research and medical application. It involves analyzing medical images, such as MRI (Magnetic Resonance Imaging) scans, to identify and classify different types of brain tumors. This field has gained significant attention due to its potential to assist in early detection, accurate diagnosis, and personalized treatment planning for brain tumor patients.

The motivation behind such classification using image data is multifold. Here are some key reasons:

1. Early Detection and Diagnosis: Early detection of brain tumors is crucial for timely intervention and improved patient outcomes. By analyzing medical images, machine learning algorithms can assist radiologists in identifying potential tumor regions that may be missed by the human eye. This can lead to earlier diagnosis, enabling prompt treatment and potentially better prognosis for patients.
</br>

2. Improved Accuracy and Objectivity: Human interpretation of medical images can be subjective and prone to errors. By leveraging machine learning techniques, the classification process can be made more objective and consistent. Algorithms can learn from large datasets, capturing intricate patterns and features that may not be immediately evident to human observers. This can enhance the accuracy and reliability of tumor classification.
</br>

3. Treatment Planning and Personalized Medicine: Different types of brain tumors may require distinct treatment approaches. Accurate classification of brain tumors based on their imaging characteristics can aid in treatment planning. It can provide valuable information about tumor size, location, invasiveness, and other important factors, enabling healthcare professionals to devise personalized treatment strategies for individual patients.
</br>

4. Research and Clinical Decision Support: Brain tumor classification using image data contributes to ongoing research efforts and the accumulation of knowledge in the field of neuro-oncology. The insights gained from analyzing large datasets can help researchers understand tumor behavior, treatment responses, and develop novel therapeutic approaches. Moreover, the use of machine learning algorithms as decision support tools can assist radiologists and oncologists in making more informed clinical decisions.
</br>

5. Automation and Efficiency: Analyzing medical images for tumor classification is a time-consuming task for radiologists and clinicians. By automating this process with machine learning models, the workload can be reduced, allowing healthcare professionals to focus on other critical aspects of patient care. This can potentially enhance overall workflow efficiency and patient management.
</br>

The dataset used for this project is obtained from Kaggle: https://www.kaggle.com/datasets/ahmedhamada0/brain-tumor-detection

We will begin by importing some useful libraries

In [ ]:
import cv2
import os
import numpy as np
from PIL import Image                                  # we use this here to convert images to RGB
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split   # for splitting dataset into train and test sets
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers import Dropout, Flatten, Dense
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.utils import normalize
from keras.models import load_model

In [ ]:
## set path to directory containing the images
image_dir = 'datasets/'

Our image directory 'datasets' consists of two sub-directories. One directory contains images with no tumors and the second those with tumors. We will now list all image files in these two directories

In [ ]:
no_tumor_images = os.listdir(image_dir +'no/')
yes_tumor_images = os.listdir(image_dir + 'yes/')

In [ ]:
# We will split our data into the dependent and independent variables and we need to create two empty lists to store these 
images = []
labels = []

# read all jpg image files in the no_tumor_images directory (and yes_tumor directory), convert them to RGB and store
# resulting image in the images list. Also store their corresponding labels in the labels list

# reading files in the no_tumor_images directory
for i, image_jpg in enumerate(no_tumor_images):
    if(image_jpg.split('.')[1]=='jpg'):
        image = cv2.imread(image_dir+'no/'+image_jpg)
        # convert image to RGB using PIL
        image = Image.fromarray(image,"RGB")
        # Resize the image to a fixed size (e.g., we will use a size of 64x64)
        image = image.resize((64, 64))
        # convert image to a numpy array and append image to images list
        images.append(np.array(image))
        # Add label to the labels list. The label in this case is 0 since the image has no tumor 
        labels.append(0)
        
# reading files in the yes_tumor_images directory
for i, image_jpg in enumerate(yes_tumor_images):
    if(image_jpg.split('.')[1]=='jpg'):
        image = cv2.imread(image_dir+'yes/'+image_jpg)
        # convert image to RGB using PIL
        image = Image.fromarray(image,"RGB")
        # Resize the image to a fixed size (e.g., we will use a size of 64x64)
        image = image.resize((64, 64))
        # convert image to a numpy array and append image to images list
        images.append(np.array(image))
        # Add label to the labels list. The label in this case is 0 since the image has no tumor 
        labels.append(1)

### Some visualization

Let's visualize some of the images in the images list

In [ ]:
plt.figure(figsize = (15,15));
for i,j in enumerate(images):
    if i<4:
        plt.subplot(1,4,i+1)
        plt.imshow(j);
        plt.xlabel(labels[i]);
        plt.tight_layout()
    else:
        break
# the x-axis of the images show the labels. Remember 0 means no tumor and 1 means the imags has a tumor

plt.savefig('images.jpg', dpi=150)
plt.show()

In [ ]:
# now, we convert the images and labels lists into numpy arrays
images = np.array(images)
labels = np.array(labels)

In [ ]:
## Split the data into train and validation sets, we split with 80% of the data as training set and the remaining 20% for
## validation
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
# let's check the shape of the train and and validation sets
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
# normalize the data
X_train=normalize(X_train, axis=1)
X_val =normalize(X_val, axis=1)

In [ ]:
# Build the CNN model (we first use a binary classifier)
model1 =  Sequential()

model1.add(Conv2D(32,(3,3),input_shape=(64,64,3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Conv2D(32,(3,3), kernel_initializer = 'he_uniform'))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Conv2D(64,(3,3), kernel_initializer = 'he_uniform'))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Flatten())
model1.add(Dense(64))
model1.add(Activation('relu'))
# Define the output layer and add droupout to prevent overfitting
# for our output layer, dense = 1 since we have a binary classification problem and for this we use a sigmoid activation funtion
# for categorical cross entropy for example dense would have been 2 and we would have used a softmax activation function
model1.add(Dropout(0.5))
model1.add(Dense(1))
# add activation function
model1.add(Activation('sigmoid'))

# Compile the model
model1.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model1_his = model1.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_val, y_val), verbose = 1,shuffle=False)

# Save the model
model1.save('BTEpochs_30.keras')

# we have a good model accuracy

In [ ]:
# Plot training & validation loss values for binary classification model
fig, ax = plt.subplots(1,2, figsize=(15,5), dpi = 300)
ax[0].plot(model1_his.history['accuracy'])
ax[0].plot(model1_his.history['val_accuracy'])
ax[0].set_title('Training and validation accuracy for binary classification model')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Val'], loc='upper left')

# Plot training & validation loss values
ax[1].plot(model1_his.history['loss'])
ax[1].plot(model1_his.history['val_loss'])
ax[1].set_title('Training and validation loss for binary classification model')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Val'], loc='upper left')

plt.tight_layout(pad =3.0) 

plt.savefig('train_ValLossBClass.jpg', dpi=150)

plt.show()

In [ ]:
# Let us also build the model using cross entropy but first, we will convert our training sets into a categorical format
y_train  = to_categorical(y_train, num_classes = 2)
y_val  = to_categorical(y_val, num_classes = 2)

model2 =  Sequential()

model2.add(Conv2D(32,(3,3),input_shape=(64,64,3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Conv2D(32,(3,3), kernel_initializer = 'he_uniform'))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Conv2D(64,(3,3), kernel_initializer = 'he_uniform'))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Flatten())
model2.add(Dense(64))
model2.add(Activation('relu'))
# output layer. Add droupout to prevent overfitting
# for our output, we now use 2 dense layer and we use the softmax activation function
model2.add(Dropout(0.5))
model2.add(Dense(2))
# add activation function
model2.add(Activation('softmax'))

# Compile the model
model2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model2_his = model2.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_val, y_val), verbose = 1,shuffle=False)

# Save the model
model2.save('BT_CEnt_Categorical_epochs_30.keras')

In [ ]:
# Plot training & validation loss values for categorical cross entropy model
fig, ax = plt.subplots(1,2, figsize=(15,5), dpi = 300)
ax[0].plot(model2_his.history['accuracy'])
ax[0].plot(model2_his.history['val_accuracy'])
ax[0].set_title('Training and validation accuracy for categorical cross entropy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Val'], loc='upper left')

# Plot training & validation loss values
ax[1].plot(model2_his.history['loss'])
ax[1].plot(model2_his.history['val_loss'])
ax[1].set_title('Training and validation loss for categorical cross entropy')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Val'], loc='upper left')

plt.tight_layout(pad =3.0) 

plt.savefig('train_ValLossCatEnt.jpg', dpi=150)
plt.show()

In [ ]:
# Evaluate the model on the test set
# as an example, we will try to predict the class of two different images, one with tumor and the other without
# but first let us load these images from the pred directory

In [ ]:
# read file 1 (with no tumor) from the test directory
image_pred_no = cv2.imread('pred/pred0.jpg')
# convert image to RGB using PIL
image_p_no = Image.fromarray(image_pred_no)
# Resize the image to a fixed size (e.g., we will use a size of 64x64)
image_p_no = image_p_no.resize((64, 64))
# convert image to a numpy array 
image_p_no = np.array(image_p_no)
# expand the dimension of the image
input_p_no = np.expand_dims(image_p_no, axis= 0)

# read file 6 (with tumor) from the test directory
image_pred_yes = cv2.imread('pred/pred5.jpg')
# convert image to RGB using PIL
image_p_yes = Image.fromarray(image_pred_yes)
# Resize the image to a fixed size (e.g., we will use a size of 64x64)
image_p_yes = image_p_yes.resize((64, 64))
# convert image to a numpy array 
image_p_yes = np.array(image_p_yes)
# expand the dimension of the image
input_p_yes=np.expand_dims(image_p_yes, axis = 0)

In [ ]:
# predict the class of the images based on the model
#result_no_model1 = model.predict_classes(image_p_no)
result_no_model1 =np.argmax(model1.predict(input_p_no),axis=1)
result_no_model2 =np.argmax(model2.predict(input_p_no),axis=1)
#result_no_model2 = model2.predict_classes(image_p_no)

result_yes_model1 =np.argmax(model1.predict(input_p_yes),axis=1)
result_yes_model2 =np.argmax(model2.predict(input_p_yes),axis=1)

Our categorical cross entropy model seem to behave better than the binary cross entropy

In [ ]:
print('N.B, if image has no tumor, label is 0, otherwise, labe is 1\n')
print('model 1 predicts image with no tumor has having label, ',result_no_model1)
print('model 2 predicts image with no tumor has having label, ',result_no_model2)
print('\n')
print('model 1 predicts image with tumor has having label, ',result_yes_model1)
print('model 2 predicts image with tumor has having label, ',result_yes_model2)

Our categorical cross entropy model seem to behave better than the binary cross entropy so we will stick with this going forward. Next we will use our model to classify all images in the test set and visualize a few of this to see how well our model performs

In [ ]:
# set directory containing test set
test_images = os.listdir('pred/')
images_test = []
images_testt = []

# reading images in the testset directory
for i, image_jpg in enumerate(test_images):
    if(image_jpg.split('.')[1]=='jpg'):
        image = cv2.imread('pred/'+image_jpg)
        # convert image to RGB using PIL
        image = Image.fromarray(image)
        # Resize the image to a fixed size (e.g., we will use a size of 128x128)
        image = image.resize((64, 64))
        # convert image to a numpy array and normalize pixel values to range between 0 and 1
        # to do this, we divide each pixel value by 255 since we have an 8-bit image
        #image = np.array(image)/255.0
        # append image to images list
        #image = np.expand_dims(image, axis= 0)
        images_test.append(np.array(image))
        images_testt.append(np.expand_dims(np.array(image), axis= 0))

In [ ]:
# make predictions
predictions = [np.argmax(model2.predict(images_testt[i]),axis=1) for i in range(len(images_testt))]

In [ ]:
# visualize the first 1o images in images_testt and add the corresponding predicted label on the x axis 
plt.figure(figsize = (15,15));
for i,j in enumerate(images_test):
    if i<10:
        plt.subplot(1,10,i+1)
        plt.imshow(j);
        plt.xlabel(predictions[i]);
        plt.tight_layout()
    else:
        break
        
plt.savefig('First_tenImages_TestSet.jpg', dpi=150)
plt.show()

In [ ]:
# visualize the first 30 images and labels
fig, ax = plt.subplots(3,10, figsize=(20,8), dpi = 500)

for i in range(10):
    ax[0,i].imshow(images_test[i])
    ax[0,i].set_xlabel(predictions[i])
    
    ax[1,i].imshow(images_test[i+10])
    ax[1,i].set_xlabel(predictions[i+10])
    
    ax[2,i].imshow(images_test[i+20])
    ax[2,i].set_xlabel(predictions[i+20])
    
plt.tight_layout()
plt.savefig('First_30Images_TestSet.jpg', dpi=150)

plt.show()